In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from utils.resolve_user_uttr import resolve_user_uttr
import pandas as pd
import time
import timeit
from tqdm.notebook import tqdm
import datetime
from zoneinfo import ZoneInfo

In [19]:
def parse_and_save_result(test_result, filename):
    def parse_clean_result(row):
        slot_filling = row["slot_filling"]
        slot_name = row["slot"]
        if (
            slot_filling
            and isinstance(slot_filling, dict)
            and slot_name in slot_filling
        ):
            row["parsed_result"] = slot_filling.get(slot_name, "")
        else:
            row["parsed_result"] = ""
        return row

    azure_openai_gpt4_result = pd.DataFrame(test_result)
    azure_openai_gpt4_result = azure_openai_gpt4_result.apply(
        parse_clean_result, axis=1
    )
    # Define the timezone for UTC+8
    timezone_utc8 = ZoneInfo("Asia/Shanghai")
    # Get the current time in UTC
    utc_now = datetime.datetime.now(ZoneInfo("UTC"))
    # Convert the UTC time to UTC+8
    local_time_utc8 = utc_now.astimezone(timezone_utc8).strftime("%Y-%m-%d_%H%M%S%z")
    azure_openai_gpt4_result.to_excel(
        f"test_results/{filename}_result_{local_time_utc8}.xlsx", index=False
    )

In [20]:
# Define the timezone for UTC+8
timezone_utc8 = ZoneInfo("Asia/Shanghai")
# Get the current time in UTC
utc_now = datetime.datetime.now(ZoneInfo("UTC"))
# Convert the UTC time to UTC+8
local_time_utc8 = utc_now.astimezone(timezone_utc8).strftime("%Y-%m-%d_%H%M%S%z")
local_time_utc8

'2024-05-28_134939+0800'

In [21]:
df = pd.read_excel("dataset/grace-nlu-chatgpt.xlsx", sheet_name="test-result")

In [22]:
df

,Unnamed: 0,idx,id,usr_uttr,grace_text,slot,parsed_result,correct_result,is_correct,idx_
0,0,1,q1_0_coughing_existance,我冇咳呀,請問你有冇咳?,cough_existence,no,no,1,1
1,1,2,q1_0_coughing_existance,冇呀,請問你有冇咳?,cough_existence,no,no,1,2
2,2,3,q1_0_coughing_existance,有呀,請問你有冇咳?,cough_existence,yes,yes,1,3
3,3,4,q1_0_coughing_existance,有,請問你有冇咳?,cough_existence,yes,yes,1,4
4,4,5,q1_0_coughing_existance,有呀，咳得好多呀,你有冇咳呀？,cough_existence,yes,yes,1,5
...,...,...,...,...,...,...,...,...,...,...
83,83,84,q2_1_sputum_amount,偶然多啲，通常都ok。,有痰，咁係多痰定少少痰？,sputum_amount,the patient experiences occasional increased s...,increased,1,84
84,84,85,q2_1_sputum_amount,早上多啲，其他時間都冇咩痰。,有痰，咁係多痰定少少痰？,sputum_amount,早上多痰，其他時間少痰。,more,1,85
85,85,86,q2_1_sputum_amount,少少痰。,有痰，咁係多痰定少少痰？,sputum_amount,the patient has experienced a small amount of ...,a little,1,86
86,86,87,q2_1_sputum_amount,好多。,有痰，咁係多痰定少少痰？,sputum_amount,the patient has experienced a lot of sputum.,a lot,1,87


In [ ]:
test_result_gpt4 = []
for id, row in tqdm(df.iterrows(), total=len(df)):
    user_utter = row["usr_uttr"]
    curr_question_id = row["id"]
    grace_utter_text = row["grace_text"]
    start_time = timeit.default_timer()
    result = resolve_user_uttr(user_utter, curr_question_id, grace_utter_text, model_type=0)
    end_time = timeit.default_timer()
    execution_time = end_time - start_time
    result["id"] = row["idx"]
    result["ground_truth"] = row["correct_result"]
    result["execution_time"] = execution_time
    result["slot"] = row["slot"]
    test_result_gpt4.append(result)

    time.sleep(1)

In [ ]:
parse_and_save_result(test_result_gpt4, "azure_gpt4")

In [23]:
# test for 4turbo model
test_result_4turbo = []
for id, row in tqdm(df.iterrows(), total=len(df)):
    user_utter = row["usr_uttr"]
    curr_question_id = row["id"]
    grace_utter_text = row["grace_text"]
    start_time = timeit.default_timer()
    result = resolve_user_uttr(
        user_utter, curr_question_id, grace_utter_text, model_type=1
    )
    end_time = timeit.default_timer()
    execution_time = end_time - start_time
    result["id"] = row["idx"]
    result["ground_truth"] = row["correct_result"]
    result["execution_time"] = execution_time
    result["slot"] = row["slot"]
    test_result_4turbo.append(result)
    time.sleep(1)

  0%|          | 0/88 [00:00<?, ?it/s]

In [24]:
# save model result for 4turbo
parse_and_save_result(test_result_4turbo, "vender_4turbo")

In [25]:
# test for 4o model
test_result_4o = []
for id, row in tqdm(df.iterrows(), total=len(df)):
    user_utter = row["usr_uttr"]
    curr_question_id = row["id"]
    grace_utter_text = row["grace_text"]
    start_time = timeit.default_timer()
    result = resolve_user_uttr(
        user_utter, curr_question_id, grace_utter_text, model_type=2)
    end_time = timeit.default_timer()
    execution_time = end_time - start_time
    result["id"] = row["idx"]
    result["ground_truth"] = row["correct_result"]
    result["execution_time"] = execution_time
    result["slot"] = row["slot"]
    test_result_4o.append(result)
    time.sleep(1)


  0%|          | 0/88 [00:00<?, ?it/s]

In [26]:
parse_and_save_result(test_result_4o, "vender_4o")